# Spatiotemporal Clustering of Land Cover Change 

The example dataset used in this project is UAV multispectral imagery collected of Niwot Ridge, Colorado collected on a weekly basis between 21 June and 18 August 2017. 

Niwot Ridge is a highly remote alpine ecosystem, exhibiting interesting dynamics between snow cover and vegetation health. In this example, I extract the Land Cover classification values from rasters from 7 different dates, treating values from each date as an indiviudal column in the training dataset, providing the temporal dimension. The latitude and longitude coordinates are also extracted to include the spatial dimension in the training data.

Running clustering this data results in the capture of significant land cover changes that can occur throughout the monitoring period. If data from the first and last date were only observed, such events wouldn't be captured or detected. Ultimately, the output shows regions where significant change has occured at a given point in time, these regions are then labeled with details on the type of change and the date ob which the change occured.

Such an approach can be applied to other time series land cover data to identify which time of year/month/week that significant deforestation is occuring for example. Another potential use case is detecting urban land use changes and localizing the area and  time of year they are typically occuring. 

1. Import Libraries
2. Prepare Analysis Files
3. Extract Raster Values for Spatiotemporal CLustering
4. Spatiotemporal Clustering with KMeans
5. Label Land Cover Change Type 
6. Vectorize Raster Data


<div>
<img src="visualization/NiwotRidge_SpatioTemporal.png" width="500"/>
</div>



### 1. Import Libraries

In [2]:
import rioxarray as rio
import os
from dask.distributed import Client, Lock
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from lc_spatiotemporal import *
import pandas as pd
from shapely.geometry import shape
import geopandas as gpd
from rasterio.features import shapes
import json


### 2. Prepare Analysis Files

In [3]:
output_dir="output"

lc_files=[f"{output_dir}/{x}" for x in os.listdir("output") if "lc" in x and x.split(".")[-1] == "tif"]
lc_files.sort()

### 3. Extract Data from Raster Time Series

In [4]:

# Instantiate the Dask Client

client = Client(n_workers=2, threads_per_worker=2, memory_limit='10GB')


data_vals=list()

xx=None
yy=None
cols=list()


for lc_file in lc_files:
   
    date=lc_file.split("/")[-1].split("_")[0]
    name=f"lc_{date}"

    # # Read Surface Temperature Band
    lc=rio.open_rasterio(lc_file, chunks=True,  lock=Lock("rio-read")) # Read Land Cover data as DaskArray

    # Add XY Coordinates to dataset for spatial dimension
    # Only add once
    if xx is None and yy is None:
        x=lc.x.values # Get X coordinates from NDVI Raster
        y=lc.y.values # Get Y coordinates from NDVI Raster

        yy, xx = np.meshgrid(y, x, indexing="ij") # Reshape to grid matching raster dimensions
        
        # Add coordinates to data list
        data_vals.append(xx) 
        cols.append("x")
        data_vals.append(yy)
        cols.append("y")

    
    # Append Land Cover values to data list
    data_vals.append(lc.isel(band=0).values)
    cols.append(name)




c:\Users\medo_\miniconda3\envs\geoai\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57803 instead
  warnings.warn(


NotImplementedError: dim is a required argument

#### 3.1 Get Raster Dimensions 

In [5]:

# Get Dimensions of the dataset to facilitate reshaping data later on
height=lc.rio.height
width=lc.rio.width
length=height * width
n_dims=len(data_vals)
orig_shape = (height,width)
data_shape = (length,n_dims)
res=lc.rio.resolution()
cell_area_m2=abs(res[0]) * abs(res[1])

### 4.1 Run KMeans Clustering on Dataset

In [6]:

N_CLUSTERS= 21


# Transpose Extracted raster values into a shape suitable for KMeans
## Typically will be reshaped to (width * height, columns) 
data=np.stack(data_vals)
data=data.transpose(1,2,0).reshape(-1, n_dims)
data_df=pd.DataFrame(data,columns=cols) # Convert array to Pandas DataFrame, use col names retrieved from the data extraction step

mask=data_df.isna().any(axis=1) # Create a mask indicating which rows have NaN 
data_df.dropna(inplace=True) # Drop NaN to prepare clustering

data_df["label"]=kmeans_df(data_df, k=N_CLUSTERS) # Run KMeans clustering on the dataframe
data_df=data_df.astype(int)



#### 4.2 Convert Labels to Raster

In [7]:
# Format Cluster Labels into XR DataArray - use NDVI dimensions to create the raster
cluster_map=np.full(length,np.nan)
cluster_map[~mask] = data_df.label.to_list()
cluster_raster=cluster_map.reshape((height, width))
cluster_da=lc.isel(band=0).copy()
cluster_da.values=cluster_raster

### 5.1 Identify Type Land Use Change 

By taking the mode (most commonly occuring) land cover class in each cluster, we can identify how land use changes over time within that cluster.

The lc_change() function detects the date at which the change in land cover occured and returns a label specifying the nature of that change, for example "Snow to Sparse Vegetation on Week 3"

The land cover change label is then mapped out to the clusters identified by KMeans, after vectorizing the dataset, we can easily visualize this data.

In [8]:
with open("output/classes.json") as classes:
    classes= json.load(classes)

class_map=dict(zip(classes.values(), classes.keys()))

vars=[x for x in data_df.columns if "lc" in x]

class_df=data_df[vars].apply(lambda x: x.map(class_map))
class_df["label"] = data_df["label"]
class_df=class_df.groupby("label").agg(pd.Series.mode).reset_index()



dates=[x.split("_")[-1] for x in vars]

lc_info_df=pd.DataFrame(class_df[vars].apply(lambda x: lc_change(x.to_list(), time_scale="Week"), axis=1).to_list())
lc_info_df["label"] = class_df.label


### 5.2 Vectorize Identified Spatio Temporal Clusters

The xr_vectorize function takes an xarray DataArray and converts it to a GeoDataframe vector polygon with matching crs.In the xr_vectorize() function you can also define a threshold for minimum number of pixels and a column name for the raster values passed to the polygon features.

Once this step is complete, the GeoDataframe is joined with the land cover change label based on the cluster label as a common index.

Lastly, the GeoDataFrame is dissolved, mergine multiple polygon geometries with the same land cover change label. This reduces the total number of polygons and can reduce processing time

In [13]:
COL_NAME="class"

# Vectorize clustered raster to polygon shapes as GeoDataFrame
gdf=xr_vectorize(raster=cluster_da, pixel_threshold=0,col_name=COL_NAME)


# Join mean values calculated for each cluster - support visualisation
gdf=gdf.set_index(COL_NAME).join(lc_info_df.set_index("label"))
gdf=gdf[~gdf.label_name.isna()]
# gdf=gdf[gdf.label_name != "No Change"]

gdf_merged=gdf.dissolve(by=COL_NAME, aggfunc="first")
gdf_merged=gdf_merged[gdf_merged.label_name != "No Change"]

# gdf.to_parquet(f"{output_dir}/kmeans_{N_CLUSTERS}_lc_25cm_.parquet")
gdf_merged.to_parquet(f"{output_dir}/kmeans_{N_CLUSTERS}_lc_25cm_merged_.parquet")

c:\Users\medo_\miniconda3\envs\geoai\Lib\site-packages\xarray\core\duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [1]:
gdf.groupby("label_name").area_m2.sum().plot.bar()

NameError: name 'gdf' is not defined